In [66]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

In [73]:
# LOAD AND STURCTURE DATA
dataset = Dataset.from_pandas(df)

train_test_split = dataset.train_test_split(test_size=0.1)
dataset = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})


def preprocess_data(examples):
    return tokenizer(examples['sentences'], padding='max_length', truncation=True, max_length=128)

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Set pad token to eos token

# Apply preprocessing
dataset = dataset.map(preprocess_data, batched=True)
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])



Map:   0%|          | 0/206766 [00:00<?, ? examples/s]

Map:   0%|          | 0/22974 [00:00<?, ? examples/s]

In [97]:


device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = AutoModelForCausalLM.from_pretrained(
    "gpt2",
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.save_pretrained('/home/maumaria/code/mmf3358/conta_tu_o_conto')


('/home/maumaria/code/mmf3358/conta_tu_o_conto/tokenizer_config.json',
 '/home/maumaria/code/mmf3358/conta_tu_o_conto/special_tokens_map.json',
 '/home/maumaria/code/mmf3358/conta_tu_o_conto/vocab.json',
 '/home/maumaria/code/mmf3358/conta_tu_o_conto/merges.txt',
 '/home/maumaria/code/mmf3358/conta_tu_o_conto/added_tokens.json',
 '/home/maumaria/code/mmf3358/conta_tu_o_conto/tokenizer.json')

In [ ]:

# FREEZE WEIGHTS
for param in model.parameters():
    param.requires_grad = False

# LoRa
config = LoraConfig(
    r=4,
    lora_alpha=32,
    lora_dropout=0.01,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


print_trainable_parameters(model)

# TRAINING
trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=500,
        learning_rate=1e-3,
        logging_steps=1,
        output_dir='outputs',
        auto_find_batch_size=True
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

torch.save(model.state_dict(), 'lora.pt')


In [ ]:
def load_model(model_path):
    model = AutoModelForCausalLM.from_pretrained(model_path)
    return model

In [100]:
trained_model = load_model('/home/maumaria/code/mmf3358/conta_tu_o_conto')
trained_tokenizer = '/home/maumaria/code/mmf3358/conta_tu_o_conto'


NameError: name 'load_model' is not defined

In [106]:
from transformers import pipeline
generator = pipeline('text-generation', model=model, tokenizer = trained_tokenizer)
generator("The White rabbit went to school", max_length=100, num_return_sequences=5)[0]['generated_text']

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


'The White rabbit went to school, you couldn\'t find the red rabbit here... We also have to find a new place for the new rabbit!" — Gunga to the rabbit in the cave.[src]\n\nGunga and Taki both worked together, and Gunga and Taki started a new school in the cave. In order to learn about the magic that had been bestowed on Kiku, Gunga used one of his own magic spells to cast the secret spell of'

In [132]:
prompt = "you are a story generator. give me a fairy tale about prince john"

   
prompt = generator(prompt, max_length=40, num_return_sequences=1, max_new_tokens=40, temperature=.6)
    
    
    
print(f"\n\n{prompt[0]['generated_text']}")   


Both `max_new_tokens` (=40) and `max_length`(=40) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




you are a story generator. give me a fairy tale about prince johnny and his love for his wife, and I'll let you know if you like it. I've got a story for you, too. I've got a story for you, too. I


In [137]:
prompt = "the prince john went to the market to buy apples"

for i in range(5):
    new_prompt = generator(prompt, max_length=40, num_return_sequences=1, max_new_tokens=40, temperature=.6)
    prompt = new_prompt[0]['generated_text']
    
print(f"\n\n\n{prompt}")

Both `max_new_tokens` (=40) and `max_length`(=40) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=40) and `max_length`(=40) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=40) and `max_length`(=40) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=40) and `max_length`(=40) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `ma




the prince john went to the market to buy apples and apples by the pound and said he saw the prince John in the car and said he was going to steal them. The prince John said no, he was going to get his apples by the pound he went to the shop and bought apples, apples and apples by the pound and there was a fight and he broke his neck and went to the hospital and said he went to the hospital to get his he had a good idea of how to get them, and he was going to steal them. He said he was going to steal them if his brother John could help him. The prince John said he he was going to steal apples and apples by the pound. The prince John said he was going to steal apples and apples by the pound. The prince John said he was going to steal apples and apples but he was going to steal the apples. The prince John said he wanted to know what John was doing.

The prince John said what John was doing is one of his most basic business transactions
